# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique:
* Model:
* Evaluation approach:
* Fine-tuning dataset:

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
# install Hugging Face Libraries
# datasets: the gateway to datasets available on huggingface_hub
! pip install --quiet "datasets==2.15.0"
#! pip install transformers[torch]
# accelerate: needed for using Trainer on PyTorch models
! pip install accelerate --upgrade
# Install the loralib library
! pip install --quiet loralib
# Install the peft - Parameter-efficient fine-tuning - for fine-tuning
# transformers, evaluate, and bitsandbytes libraries, upgrading them if they are already installed
! pip install "peft" "transformers" "evaluate" "bitsandbytes" --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━

In [2]:
# Run the command 'nvidia-smi' and store the output in the variable 'gpu_info'
gpu_info = !nvidia-smi

# Join the lines of the output into a single string, separated by newline characters
gpu_info = '\n'.join(gpu_info)

# Check if the string 'failed' is present in the 'gpu_info' string
if gpu_info.find('failed') >= 0:
    # If 'failed' is found, print a message indicating that the GPU is not connected
    print('Not connected to a GPU')
else:
    # If 'failed' is not found, print the 'gpu_info' string containing GPU information
    print(gpu_info)

Mon Apr  8 07:19:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   44C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import torch
# check availability of the GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print('Current Device: ', device)

Current Device:  cuda


In [4]:
# Import the datasets packages
from datasets import load_dataset

# Load the rotten_tomatoes dataset, containing movie reviews
# so we use the train_test_split method to split it into train and test
dataset = load_dataset("rotten_tomatoes", split="train").train_test_split(
    test_size=0.2, shuffle=True, seed=23
)

splits = ["train", "test"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [5]:
# View the training dataset structure
dataset["train"]

Dataset({
    features: ['text', 'label'],
    num_rows: 6824
})

In [6]:
# show sizes
print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

Train dataset size: 6824
Test dataset size: 1706


In [7]:
# looking at the first example
# and stating if it is positive or negative
dataset["train"][0]

{'text': 'a punch line without a premise , a joke built entirely from musty memories of half-dimensional characters .',
 'label': 0}

In [8]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, EvalPrediction

# selection of the GPT2 version model
model_id="gpt2"

# define the tokenizer to fine-tune
tokenizer = GPT2Tokenizer.from_pretrained(model_id)
#tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>') #gpt2-medium
# set the pad token
tokenizer.pad_token = tokenizer.eos_token

# load the GPT2 pre-trained model for classification
model = GPT2ForSequenceClassification.from_pretrained(model_id, num_labels=2)
# solve issue with CUDA
model.resize_token_embeddings(len(tokenizer))
# set the pad token of the model's configuration
model.config.pad_token_id = model.config.eos_token_id
#model = model.to(device)
# OR model = GPT2ForSequenceClassification(configuration).from_pretrained(model_name).to(device)
# set the pad token of the model's configuration
#model.config.pad_token_id = model.config.eos_token_id

# Let's use a lambda function to tokenize all the examples
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["text"], padding="max_length",truncation=True), batched=True
    )

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/6824 [00:00<?, ? examples/s]

Map:   0%|          | 0/1706 [00:00<?, ? examples/s]

In [9]:
print("The max model length is {} for this model, although the actual embedding size for GPT small is 768".format(tokenizer.model_max_length))
print("The beginning of sequence token {} token has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.bos_token_id), tokenizer.bos_token_id))
print("The end of sequence token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.eos_token_id), tokenizer.eos_token_id))
print("The padding token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.pad_token_id), tokenizer.pad_token_id))

The max model length is 1024 for this model, although the actual embedding size for GPT small is 768
The beginning of sequence token <|endoftext|> token has the id 50256
The end of sequence token <|endoftext|> has the id 50256
The padding token <|endoftext|> has the id 50256


In [10]:
# Show the first example of the tokenized training set
print(tokenized_dataset["train"][0]["input_ids"])

[64, 10862, 1627, 1231, 257, 18659, 837, 257, 9707, 3170, 5000, 422, 1276, 88, 9846, 286, 2063, 12, 19577, 3435, 764, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 

In [11]:
# show the model
print(model)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)


In [12]:
# Freeze all the parameters
for param in model.parameters():
  param.requires_grad = False

# Unfreeze the last layer/some layers
for param in model.score.parameters():
    param.requires_grad = True

In [13]:
# hyperparameters
batch_size = 4
lr = 1e-3
num_epochs = 1

In [14]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

import numpy as np
# function definition for computing the metric
def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='weighted')
    return {"accuracy": accuracy_score(p.label_ids, preds), "f1": f1, "precision": precision, "recall": recall}

In [15]:
# Use Hugging Face’s Trainer
# to fine-tune the model
# import:
# - the class for handling padding,
# - the class for training and evaluation,
# - the class for defining the training arguments
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=True,  # Enable mixed precision training
    gradient_accumulation_steps=2,  # Example: accumulate gradients over 2 steps
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer), #specify the data collator
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [16]:
# evaluate the model performance on the test set
# prior fine tuning
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 4.509130001068115,
 'eval_accuracy': 0.5105509964830012,
 'eval_f1': 0.3451221714681366,
 'eval_precision': 0.26066232000978545,
 'eval_recall': 0.5105509964830012,
 'eval_runtime': 102.1865,
 'eval_samples_per_second': 16.695,
 'eval_steps_per_second': 2.094}

In [17]:
# DELETE fine tune the model
# trainer.train()

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [18]:
from peft import LoraConfig, get_peft_model, TaskType

# instatiation of the class LoraConfig - Low Rank Adaptation for PEFT
lora_config = LoraConfig(
    r=4,
    lora_alpha=16, # strength of regularization
    lora_dropout=0.1, # regularization
    bias="none", # no biais
    #task_type="CAUSAL_LM",
    task_type=TaskType.SEQ_CLS, # sequence classification
    # tune the embedding layer and prediction head
    #modules_to_save=["wte", "lm_head"]
    inference_mode=False # use for both training and inference
)

# get a trainable PEFT model (using the LoRA config instantiated previously)
lora_model = get_peft_model(model, lora_config)

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1059: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [19]:
# Checking Trainable Parameters of a PEFT Model
lora_model.print_trainable_parameters()

trainable params: 148,992 || all params: 124,590,336 || trainable%: 0.11958551905663052


In [20]:
# hyperparameters
lr = 1e-3
batch_size = 4
num_epochs = 1

In [21]:
training_args = TrainingArguments(
    output_dir="./gpt-2-lora",
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [22]:
# Pass the model, training arguments, dataset, tokenizer,
# and any other necessary component to the Trainer, and call train to start training
gpt_peft_trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"], # training dataset requires column input_ids
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer), #specify the data collator
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [23]:
gpt_peft_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.617200,0.554072,0.737984,0.737449,0.738812,0.737984


TrainOutput(global_step=1706, training_loss=0.8562847776960796, metrics={'train_runtime': 871.1544, 'train_samples_per_second': 7.833, 'train_steps_per_second': 1.958, 'total_flos': 3572424788410368.0, 'train_loss': 0.8562847776960796, 'epoch': 1.0})

In [24]:
# only saves the adapter weights
lora_model.save_pretrained("./data/sav/gpt-lora")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [25]:
from peft import PeftModelForSequenceClassification, PeftConfig

In [26]:
from transformers import BitsAndBytesConfig, AutoModelForSequenceClassification

dir_sav = "./data/sav/gpt-lora"
#config = PeftConfig.from_pretrained(dir_sav)

# quantization in 4-bit format to reduce the memory footprint compared to the 32-bit format
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [27]:
inference_model = AutoModelForSequenceClassification.from_pretrained(
    dir_sav,
    device_map='auto',
    quantization_config=bnb_config,
    use_cache=False
)
inference_model.config.pad_token_id = inference_model.config.eos_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
# Pass the model, training arguments, dataset, tokenizer,
# and any other necessary component to the Trainer, and call train to start training
infer_peft_trainer = Trainer(
    model=inference_model,
    args=training_args,
    eval_dataset=tokenized_dataset["test"],
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer), #specify the data collator
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [29]:
# Show the performance of the model
infer_peft_trainer.evaluate()

{'eval_loss': 0.5589032173156738,
 'eval_accuracy': 0.7303634232121923,
 'eval_f1': 0.7290953964051652,
 'eval_precision': 0.7329111696183853,
 'eval_recall': 0.7303634232121923,
 'eval_runtime': 136.8249,
 'eval_samples_per_second': 12.468,
 'eval_steps_per_second': 3.121}